In [4]:
import numpy as np
from math import pi, cos, sin, radians
import time

### Forward Kinematic (Vorherige Aufgabe)

In [6]:
def forward_kinematic_braccio(joint_angles):
    ''' param: joints: current position of all the joints of the braccio '''
    ''' return: current position'''

    result = np.eye(4)
    
    ## initial translation
    result = result @ translation([281, 0, 3])
    
    ## Joint 0 (Base b)
    result = result @ rotate_z(90 - joint_angles[0]) @ translation([0, 0, 71]) 
    
    ## Joint 1 (Shoulder s)
    result = result @ rotate_y(90 - joint_angles[1]) @ translation([0, 0, 126]) 
    
    ## Joint 2 (Elbow e)
    result = result @ rotate_y(90 - joint_angles[2]) @ translation([0, 0, 124]) 
    
    ## Joint 3 (Wrist tilt v)
    result = result @ rotate_y(90 - joint_angles[3]) @ translation([0, 0, 60])

    ## Joint 4 (Wrist rotate w)
    result = result @ rotate_z(90 - joint_angles[4]) @ translation([0, 0, 134]) 
    
    return result

def forward_kinematic_braccio_each_joint(joint_angles):
    ''' param: joints: current position of all the joints of the braccio '''
    ''' return: current position'''

    result = np.eye(4)
    positions = []
    
    ## initial translation
    result = result @ translation([281, 0, 3])
    positions.append(result[0:3,3])
    
    ## Joint 0 (Base b)
    result = result @ rotate_z(90 - joint_angles[0]) @ translation([0, 0, 71]) 
    positions.append(result[0:3,3])
    
    ## Joint 1 (Shoulder s)
    result = result @ rotate_y(90 - joint_angles[1]) @ translation([0, 0, 126]) 
    positions.append(result[0:3,3])
    
    ## Joint 2 (Elbow e)
    result = result @ rotate_y(90 - joint_angles[2]) @ translation([0, 0, 124]) 
    positions.append(result[0:3,3])
    
    ## Joint 3 (Wrist tilt v)
    result = result @ rotate_y(90 - joint_angles[3]) @ translation([0, 0, 60])
    positions.append(result[0:3,3])

    ## Joint 4 (Wrist rotate w)
    result = result @ rotate_z(90 - joint_angles[4]) @ translation([0, 0, 134]) 
    positions.append(result[0:3,3])
    
    return positions


def translation(trans):
    return np.array([
        [1,0,0,trans[0]],
        [0,1,0,trans[1]],
        [0,0,1,trans[2]],
        [0,0,0,1]
    ])

def rotate_z(theta):
    rad = radians(theta)
    return np.array([
        [cos(rad), -sin(rad), 0, 0],
        [sin(rad), cos(rad), 0, 0],
        [0,0,1,0],
        [0,0,0,1]
    ])

def rotate_y(theta):
    rad = radians(theta)
    return np.array([
        [cos(rad),0, sin(rad), 0],
        [0,1,0,0],
        [-sin(rad),0, cos(rad), 0],
        [0,0,0,1]
    ])

def forward_kinematic_braccio_fixed(joint_angles):
    b = joint_angles[0]
    s = 110
    e = joint_angles[1]
    v = joint_angles[2]
    w = 90
    g = 73
    joint_angles_with_fixed = [b, s, e, v, w, g]
    return forward_kinematic_braccio(joint_angles_with_fixed)

### Plot linear movement

In [2]:
import plotly.graph_objects as go
import pandas as pd

[array([281.,   0.,   3.]),
 array([281.,   0.,  74.]),
 array([241.94308458, -18.21253877, 192.40127022]),
 array([129.98856705, -70.41778765, 203.20858232]),
 array([111.3900359 , -79.09042516, 146.82702507]),
 array([ 69.85331633, -98.45931559,  20.90821389])]

In [11]:
data = pd.DataFrame(positions, columns = ['x','y','z'])

In [8]:
positions_braccio = forward_kinematic_braccio_each_joint([65, 110, 155, 165, 90, 73])
data_braccio = pd.DataFrame(positions_braccio, columns = ['x','y','z'])
data_braccio.head()

,x,y,z
0,281.000000,0.000000,3.000000
1,281.000000,0.000000,74.000000
2,241.943085,-18.212539,192.401270
3,129.988567,-70.417788,203.208582
4,111.390036,-79.090425,146.827025


In [63]:
for i in range(1, 7):
    braccio_plot = go.Scatter3d(x=data_braccio['x'][:i], y=data_braccio['y'][:i], z=data_braccio['z'][:i], name='Braccio')

    pdata = [braccio_plot]

    layout = go.Layout(
        scene = dict(
            camera=dict(
                up=dict(x=0, y=0, z=1),
                center=dict(x=0, y=0, z=0),
                eye=dict(x=-1.3, y=-1.7, z=0.06)
            ),
            xaxis = dict(range = [0,400]),
            yaxis = dict(range = [-150,150]),
            zaxis = dict(range = [0,265], backgroundcolor="rgba(205,133,63,1)"),
            aspectmode='cube'
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )

    fig = go.Figure(data=pdata, layout=layout)
    fig.update_traces(marker=dict(
            color='red'))
    fig.write_image(f'joint{i}.svg')